# 导入相关库 

In [1]:
import numpy as np
import pandas as pd
import re
import jieba
import os

# 导入数据

In [2]:
data = pd.read_csv('./datas/data_clean.csv',sep='##',names=['from','to','date','content','label'],encoding='utf8',engine='python')
data.head()

,from,to,date,content,label
0,"""yan""<(8月27-28,上海)培训课程>",lu@ccert.edu.cn,"Tue, 30 Aug 2005 10:08:15 +0800",非财务纠淼牟莆窆芾-（沙盘模拟）------如何运用财务岳硖岣吖芾砑ㄐ[课 程 背 景]每一...,1
1,"""pan"" <pan@jdl.ac.cn>",shi@ccert.edu.cn,"Sun, 14 Aug 2005 10:16:47 +0800",讲的是孔子后人的故事。一个老领导回到家乡，跟儿子感情不和，跟贪财的孙子孔为本和睦。老领导的弟...,0
2,=?GB2312?B?1cW6o8TP?= <jian@163.con>,xing@ccert.edu.cn,"Sun, 14 Aug 2005 10:17:57 +0800",尊敬的贵公司(财务/经理)负责人您好！我是深圳金海实业有限公司（广州。东莞）等省市有分公司。...,1
3,=?GB2312?B?tPq/qreixrE=?= <pan@12.com>,ling@ccert.edu.cn,"Sun, 14 Aug 2005 10:19:02 +0800",贵公司负责人(经理/财务）您好：深圳市华龙公司受多家公司委托向外低点代开部分增值税电脑发票（...,1
4,"""mei"" <mei@dghhkjk.com>",tang@ccert.edu.cn,"Sun, 14 Aug 2005 10:21:22 +0800",这是一封HTML格式信件！---------------------------------...,1


# 数据处理

## 收件邮箱与发件邮箱处理

In [3]:
def getEmail(row):
    li = re.findall('@([A-Za-z0-9]+\.[A-Za-z0-9\.]+)',str(row))
    if len(li)>0:
        return li[0]
    else:
        return 'unknown'

data['from'] = data['from'].apply(getEmail)
data['to'] = data['to'].apply(getEmail)

## 时间处理

In [4]:
data['date'] = data['date'].astype('str')

In [5]:
dateLength = data['date'].map(len).sort_values().unique() # 获取不同时间格式的长度
dates = data['date'].unique() # 获取唯一时间
for length in dateLength:
    print(length, list(filter(lambda row:len(row)==length,dates))[0]) # 如果长度相等，获取时间

3 nan
16 2005-9-2 上午11:04
20 Sep 23, 2005 1:04 AM
23 August 24, 2005, 5:00pm
24 Thu, 1 Sep 2005 09:42:01
25 Mon, 15 Aug 2005 07:04:08
27 Sat, 1 Oct 2005 00:12:07 UT
28 Mon, 1 Jan 2001 21:40:47 GMT
29 Sun, 14 Aug 2005 11:59:22 GMT
30 Thu, 3 Feb 2005 04:03:08 +0800
31 Tue, 30 Aug 2005 10:08:15 +0800
32 Tue, 16 Aug 2005 06:19:16 中国标准时间
33 Mon,2TThu, 1 Sep 2005 01:41:29 PM
34 Wed,2FSun, 28 Aug 2005 03:33:18 PM
35 Thu,1Fri, 2 Sep 2005 09:02:56 +0800
36 Thu, 1 Sep 2005 06:28:02 +0800 (CST)
37 Sun, 14 Aug 2005 13:05:41 +0800 (CST)
46 Sun, 2 Oct 2005 04:18:16 -Mailer: OpenSmtp.net
47 Sun, 14 Aug 2005 10:21:23 -Mailer: OpenSmtp.net
58 Thu, 1 Sep 2005 03:46:34  +0800  X-Priority:  3  X-Mailer:
59 Mon, 12 Sep 2005 11:16:56  +0800  X-Priority:  2  X-Mailer:
62 Thu, 1 Sep 2005 20:09:06 +0800 X-Priority: 3 X-Mailer: FoxMail
63 Wed, 31 Aug 2005 00:02:40 +0800 X-Priority: 3 X-Mailer: FoxMail


In [6]:
def func(row):
    length = len(row)
    hour = '' # 经过观察，判断是否是垃圾邮件，最关键的是小时，和年份，月份，天数，星期无关
    temp = 0 # 临时变量
    
    if length == 3:
        return np.nan
    elif length == 16:
        if '下午' in row:
            temp = 12
        return int(re.findall('(\d+):\d+', row)[0])+temp
    elif length == 20:
        if 'PM' in row: 
            temp = 12
        return int(re.findall('(\d+):\d+', row)[0])+temp
    elif length == 23:
        if 'pm' in row: 
            temp = 12
        return int(re.findall('(\d+):\d+', row)[0])+temp
    else:
        return int(re.findall('(\d{2}):\d{2}:\d{2}', row)[0])

data['date'] = data['date'].apply(func)

## 内容特征处理

In [7]:
def func(row):
    return ' '.join(jieba.cut(row))

data['content'] = data['content'].astype('str')
data['content'] = data['content'].apply(func)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\zylin\AppData\Local\Temp\jieba.cache
Loading model cost 0.937 seconds.
Prefix dict has been built succesfully.


In [8]:
data.head()

,from,to,date,content,label
0,unknown,ccert.edu.cn,10.0,非 财务 纠淼 牟 莆 窆 芾 - （ 沙盘 模拟 ） - - - - - - 如何 运用 ...,1
1,jdl.ac.cn,ccert.edu.cn,10.0,讲 的 是 孔子 后人 的 故事 。 一个 老 领导 回到 家乡 ， 跟 儿子 感情 不 和...,0
2,163.con,ccert.edu.cn,10.0,尊敬 的 贵 公司 ( 财务 / 经理 ) 负责人 您好 ！ 我 是 深圳 金海 实业 有限...,1
3,12.com,ccert.edu.cn,10.0,贵 公司 负责人 ( 经理 / 财务 ） 您好 ： 深圳市 华龙 公司 受 多家 公司 委托...,1
4,dghhkjk.com,ccert.edu.cn,10.0,这是 一封 HTML 格式 信件 ！ - - - - - - - - - - - - - -...,1


In [9]:
if not os.path.exists('./datas'):
    os.mkdir('./datas')
    
data.to_csv('./datas/data_pre.csv',index=False)